In [1]:
from langchain import PromptTemplate #用于 PromptTemplate 为字符串提示创建模板
from langchain.prompts.pipeline import PipelinePromptTemplate #for multiple prompt merging

#base class of all prompt generators, provide single prompt given arguments
class PromptGenerator:
    def __init__(self, promptFString):
        self.prompt_template = PromptTemplate.from_template(
            promptFString
            #Sample: "Tell me a {adjective} joke about {content}."
        )
    
    def generatePrompt(self,**kwargs):
        formatted_prompt = self.prompt_template.format(**kwargs)
        return formatted_prompt


#prompt generator for MAD situation:
class MADPromptGenerator:
    def __init__(self, prePromptGuide, postPromptGuide, numOfAgents):
        self.prePromptGuide = PromptTemplate.from_template( prePromptGuide)
        self.postPromptGuide = PromptTemplate.from_template(postPromptGuide)
        self.numOfAgents = numOfAgents
        self.angentsAnsGuide = PromptTemplate.from_template("\n\n".join( [f" One agent response:{{example{index}}}" for index in range(numOfAgents)]))

    def generatePrompt(self, prevAgentsInputs, currentAgentIndex,currentRound, preInputs=None,postInputs=None):
        if preInputs !=None:
            prePrompt = self.prePromptGuide.format(preInputs)
        else:
            prePrompt =  self.prePromptGuide.format()
        
        if postInputs != None:
            postPrompt = self.postPromptGuide.format(postInputs)
        else:
            postPrompt = self.postPromptGuide.format()
        
        
        fullPrompt = prePrompt + "\n \n"
        for i in range(currentRound):
            currentAgentInput = prevAgentsInputs[i]["example{}".format(currentAgentIndex)]
            fullPrompt = fullPrompt +  f" {currentAgentInput}" + "\n\n"
            prevAgentsInputs[i]["example{}".format(currentAgentIndex)] = " "
            currentRoundAgentsContext = self.angentsAnsGuide.format(**(prevAgentsInputs[i]))
            fullPrompt = fullPrompt + "These are the recent/updated opinions from other agents:"+ currentRoundAgentsContext + "\n \n"
            prevAgentsInputs[i]["example{}".format(currentAgentIndex)] = currentAgentInput
        
        fullPrompt += postPrompt

        return fullPrompt
    

In [2]:
#return type definition
from pydantic import BaseModel, Field

class MathAns(BaseModel):
    DetailedExplainationsToConvinceOthers: str = Field(description="detailed record of explainations, should be as detailed and lengthy (at least 100 words) as possible in order to convince others")
    answer: float = Field(description="Final Answer")

In [3]:
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser
import numpy as np
import os 

import time

openaiApiKey = ''
if openaiApiKey is None:
    raise ValueError("OpenAI API key not found. Make sure you have an .env file with the key defined.")
model_name ="gpt-3.5-turbo-0301" #"gpt-3.5-turbo-instruct"#"gpt-3.5-turbo"# "text-davinci-003"

class ProcessHandler:
    def __init__(self,promptFString):
        self.pG = PromptGenerator(promptFString)
        self.llm = OpenAI(temperature=0.1, openai_api_key=openaiApiKey,model_name=model_name)
    
    def generateAnswer(self, questionPromptString):
        formatedQuestion = self.pG.generatePrompt(questionPromptString)
        return self.llm(formatedQuestion)
    
class MADProcessHandler:
    def __init__(self,nrounds,nAgents, prePromptString, postPromptString, parser):
        self.pG = MADPromptGenerator(prePromptString,postPromptString,nAgents)
        self.nrounds = nrounds
        self.llm = OpenAI( openai_api_key=openaiApiKey,model_name=model_name)
        self.answersRecords = []
        self.nAgents = nAgents
        self.parser = parser

    def generateAnswer(self, questionSting, prePromptString=None, postPromptString=None):
        self.answersRecords.clear()
        if prePromptString != None:
            questionSting += prePromptString
        for i in range(self.nrounds):
            self.answersRecords.append({})
            for j in range(self.nAgents):
                prompt = questionSting
                if i !=0:
                    print(i)
                    prompt = prompt + self.pG.generatePrompt(self.answersRecords, j,i)
                prompt = prompt+self.parser.get_format_instructions()
                print("round{}".format(i), "agent{} prompt: \n".format(j),prompt)
                answer = self.llm(prompt)
                print("round{}".format(i), "agent{} answer: \n".format(j),answer)
                time.sleep(30)
                self.answersRecords[i]["example{}".format(j)] = answer
        
        prompt = questionSting+"\n\n" + "The following paragraphs are previous answers given by other agents for your reference \n\n"
        for j in range(self.nAgents):
            prompt = prompt+ "agent{}'s answer:\n".format(j)+self.answersRecords[-1]["example{}".format(j)]+"\n\n"
        prompt = prompt + "Given the answers from other agents, please draw a final conclusion. Please Think Step by Step. Make sure to state your answer at the end of the response."

        prompt = prompt+"\n"+self.parser.get_format_instructions()
        answer = self.llm(prompt)
        
        answer = self.parser.parse(answer)
        return answer

In [4]:
questions = []
correct_answer = []
#question number 1
questions.append('''A boat is acted on by a river current flowing north and by wind blowing on its sails. The boat travels northeast. In which direction is the wind most likely applying force to the sails of the boat?''') #Mercury_7041860
correct_answer.append('East')
#question number 2
questions.append('''In a group of 10 people, each one has 7 friends among the other nine people. Prove that there exist 4 people who are friends of each other.''')
correct_answer.append('''Consider an edge between two people means friendship between
them. Every node is connected to 7 other nodes. Consider two
nodes A, B that are already friend of each other. Out of the remain-
ing 8 people, A, B are each connected to 6 of them. So based on
pigeonhole principle, They have at least 4 mutual friends. Consider
this 4 mutual friends to be x1, x2, x3, x4. If there exists any edge
between any pair of them we are done because we will have 4 people
who are pairwise friend of each other. So there must be no edge
among any pair of them. Now consider any of those nodes. It has
to be connected to 7 nodes. However, Now maximum degree of that
node is 6 as it can not be connected to 3 nodes (and itself ), which is
a contradiction. So it means there exists 4 people that are pairwise
friend of each other.
''')

No startegy Test

In [5]:
#no strategy test

llm = OpenAI(temperature=0.5, openai_api_key=openaiApiKey,model_name=model_name)

for question in questions:
    prompt = question       
    ans = llm(prompt)


/Users/ronaldleung/miniconda3/envs/MAD/lib/python3.11/site-packages/langchain_community/llms/openai.py:248: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


ValidationError: 1 validation error for OpenAIChat
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

Using CoT

In [ ]:

llm2 = OpenAI(temperature=0.5, openai_api_key=openaiApiKey,model_name=model_name)

for question in questions:
    prompt = question + "Please think step by step."    
    ans = llm(prompt)

Using MAD 

In [ ]:
agents = 2
rounds = 2

parser = PydanticOutputParser(pydantic_object=MathAns)
ph = MADProcessHandler(rounds,agents,
     " Make sure to state your answer at the end of the response."
     ,"Use these opinions carefully as additional advice, can you provide an updated answer? Make sure to state your answer at the end of the response.",
    parser)

for question in questions:
    prompt = question + "Please think step by step."    
    ans = ph.generateAnswer(prompt)